In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/text-corpus/text_corpus.txt


In [2]:
import tensorflow.keras
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import LSTM,Dense,Flatten,Embedding
from keras.preprocessing.sequence import pad_sequences
import re
from nltk.corpus import stopwords
import string

import joblib

In [3]:
file=open("../input/text-corpus/text_corpus.txt",'r')
text=file.read()
file.close()
print(text[0:100])

A Game Of Thrones 
Book One of A Song of Ice and Fire 
By George R. R. Martin 
PROLOGUE 
"We should 


In [4]:
def clean_text(text):
    sample=text
    sample=re.sub('[%s]'% re.escape(string.punctuation),'',sample)
    sample=[word for word in sample.split() if word.isalpha()]
    sample=[word.lower() for word in sample]
    sample=' '.join(sample)
    return sample

In [5]:
cleaned_text=clean_text(text)
cleaned_text[0:100]
print(len(cleaned_text.split()))

292883


In [6]:
seq_doc=[]
seq_len=50
l=seq_len+1
tokens=[w for w in cleaned_text.split()]
for i in range(l,len(tokens)):
    seq=tokens[i-l:i]
    line=' '.join(seq)
    seq_doc.append(line)
    

In [7]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(seq_doc)
sequences=tokenizer.texts_to_sequences(seq_doc)
vocab_size=len(tokenizer.word_index)+1

In [8]:
sequences=np.array(sequences)
X,Y=sequences[:,:-1],sequences[:,-1]
Y=to_categorical(Y,num_classes=vocab_size)
seq_len=X.shape[1]

In [9]:
print(vocab_size,seq_len)

11924 50


In [10]:
def model(vocab_size,seq_length):
    model=Sequential()
    model.add(Embedding(vocab_size,50,input_length=seq_length))
    model.add(LSTM(100,return_sequences=True))
    model.add(LSTM(100))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(vocab_size,activation='softmax'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [11]:
model=model(11924,50)

2022-11-13 17:05:11.232338: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            596200    
_________________________________________________________________
lstm (LSTM)                  (None, 50, 100)           60400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 11924)             1204324   
Total params: 1,951,424
Trainable params: 1,951,424
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(X,Y,batch_size=128,epochs=10)

2022-11-13 17:05:31.159699: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
2288/2288 [==============================] - 402s 174ms/step - loss: 6.4725 - accuracy: 0.0691
Epoch 2/10
2288/2288 [==============================] - 399s 174ms/step - loss: 5.9396 - accuracy: 0.0951
Epoch 3/10
2288/2288 [==============================] - 401s 175ms/step - loss: 5.6218 - accuracy: 0.1141
Epoch 4/10
2288/2288 [==============================] - 401s 175ms/step - loss: 5.4063 - accuracy: 0.1242
Epoch 5/10
2288/2288 [==============================] - 406s 177ms/step - loss: 5.2662 - accuracy: 0.1298
Epoch 6/10
2288/2288 [==============================] - 406s 178ms/step - loss: 5.1283 - accuracy: 0.1358
Epoch 7/10
2288/2288 [==============================] - 406s 177ms/step - loss: 5.0146 - accuracy: 0.1402
Epoch 8/10
2288/2288 [==============================] - 404s 176ms/step - loss: 4.9161 - accuracy: 0.1440
Epoch 9/10
2288/2288 [==============================] - 401s 175ms/step - loss: 4.8471 - accuracy: 0.1466
Epoch 10/10
2288/2288 [=======================

In [13]:
model.save('text_gen_model.h5')
joblib.dump(tokenizer,open('tokenizer_text_gen.joblib','wb'))

In [14]:
def generate_seq(model,tokenizer,seq_length,seed_text,n_words):
    result=list()
    in_text=seed_text
    for i in range(n_words):
        encoded=tokenizer.texts_to_sequences([in_text])[0]
        encoded=pad_sequences([encoded],maxlen=seq_length,truncating='pre')
        y=model.predict(encoded,verbose=0)
        ymax=np.argmax(y,axis=1)
        out_word=''
        for word,index in tokenizer.word_index.items():
            if index==ymax:
                out_word=word
                break
        in_text=in_text+' '+out_word
        result.append(out_word)
    return ' '.join(result)

In [15]:
seed_text=seq_doc[np.random.randint(0,len(seq_doc))]
print(seed_text+'\n')
print(generate_seq(model,tokenizer,seq_len,seed_text,50))

you back your homes or your crops nor can i restore your dead to life but perhaps i can give you some small measure of justice in the name of our king robert every eye in the hall was fixed on him waiting slowly ned struggled to his feet pushing himself

the paper and the wall was a man of the seven kingdoms and the sun had been a man in the wall and the sun of the wall and the sun of the world and the others were a man in the north the king had been a man in
